In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

**Changes since Tues. 4/23:**

- new column for pre-test average policy support (to use as control in model)
- clarified order for treatment values; treatment value/frame mapping switched around a bit
- Lin estimator function from shared document being used for regression
- first pass at regression analysis (effect of treatment conditions, demeaned covariates and interactions between them on (post-test) average policy support, treating pre-test response as covariate) with robust covariance/standard errors (HC3)
- individually demeaned covariate columns from before are now removed since demeaning process is wrapped in the Lin estimator function

**Changes since Fri. 4/26:**

- using `statsmodels.formula.api` instead of `statsmodels.api` but it makes no difference
- output for `model1` regression results checked with R `lm_lin()` output -- there are slight differences but it has to do with how `lm_lin()` takes the mean of covariates

In [2]:
pilot_data = pd.read_csv("../data/pilot_data.csv", skiprows=[1, 2])

In [3]:
data = pilot_data[['GasTax', 'CarbTax',
       'Treaty', 'RegCarb', 'political_views', 'party_id', 'party_id.1',
       'party_id.2', 'QID74', 'ScientificConfidence', 'RewardConsequence ',
       'Attention_Check_1', 'Religiosity', 'Economic_Reasoning',
       'Attention_Check_2', 'prosociality_1', 'prosociality_2',
       'prosociality_3', 'prosociality_4', 'prosociality_5', 'prosociality_6',
       'prosociality_7', 'prosociality_8', 'prosociality_9', 'GasTax_after',
       'CarbTax_after', 'Treaty_after', 'RegCarb_after', 'treatment_value',
       "ResponseId"]]

In [4]:
# filter responses based on 2 attention checks
data = data.loc[(data["Attention_Check_1"] == "Strongly like") &
                (data["Attention_Check_2"] == '1,3'), :]

In [5]:
# 186 observations
data.shape, pilot_data.shape

((186, 30), (202, 52))

In [6]:
# method 1: "main_party_id" -- consolidate Independent and No preference (should ask the other data group)
data.loc[:, "main_party_id"] = data["party_id"]
data.loc[(data["party_id"] == "Independent") | (data["party_id"] == "No preference"), "main_party_id"] = "Independent_nopref"

In [7]:
data.main_party_id.value_counts()

main_party_id
Democrat              86
Independent_nopref    72
Republican            28
Name: count, dtype: int64

In [8]:
# method 2: "party" -- group by Democrat/Republican-leaning, then include or exclude pure Independents/no preference
data.loc[(data["party_id"] == "Democrat") | (data["QID74"] == 2), "party"] = "D"
data.loc[(data["party_id"] == "Republican") | (data["QID74"] == 4), "party"] = "R"
data.loc[(data["QID74"] == 3), "party"] = "I"

In [9]:
data.party.value_counts()

party
D    117
R     37
I     32
Name: count, dtype: int64

In [10]:
# average policy support (in [0, 3])
data["avg_policy_support"] = data[['GasTax_after', 'CarbTax_after',
                                   'Treaty_after', 'RegCarb_after']].mean(axis=1)

In [11]:
# mapping treatment values to treatment condition names
treatments = {0: "No framing",
              1: "Positive science",
              2: "Negative science",
              3: "Religious",
              4: "Equity",
              5: "Efficiency",
              6: "Secular"}
data["treatment_frame"] = data["treatment_value"].map(treatments)

In [12]:
# distribution of subjects across treatment conditions (like Table 1 from paper)
# N = 186
treatment_freq = data[["treatment_value", "treatment_frame"]].value_counts()
treatment_rel_freq = data["treatment_frame"].value_counts(normalize=True)
treatment_freq.to_frame().sort_index().join(treatment_rel_freq)

,,count,proportion
treatment_value,treatment_frame,,
0,No framing,22,0.118280
1,Positive science,22,0.118280
2,Negative science,18,0.096774
3,Religious,35,0.188172
4,Equity,30,0.161290
5,Efficiency,32,0.172043
6,Secular,27,0.145161


**1. What is the mean response under each of the different framings, on average, and separately for dems/republicans?**

Difference in means

- A simple table of the average climate policy support under each of the different framings, with standard errors.  
- Average policy support for different framing separated by political position

Regression analysis

- Basic regression analysis of framing’s impact on policy position, controlling for de-meaned covariates and de-meaned covariates + treatment interactions. Use robust standard errors.
- Include pre-test response as a control, and list other controls based on the data quality group's coding. 
- We should use the Lin estimator, as we did in week 3, where we de-mean all covariates, and then control for covariates and covariate-treatment interactions.

In [13]:
pd.pivot_table(data, values=["avg_policy_support"],
               index=["treatment_value","treatment_frame"],
               aggfunc=['mean', 'sem'])

,,mean,sem
,,avg_policy_support,avg_policy_support
treatment_value,treatment_frame,,
0,No framing,1.590909,0.167378
1,Positive science,1.795455,0.186559
2,Negative science,1.972222,0.238192
3,Religious,1.857143,0.109566
4,Equity,1.916667,0.115801
5,Efficiency,1.953125,0.137224
6,Secular,1.990741,0.124809


In [14]:
# temporarily using method 2 (Independents are D/R-leaning or pure Independent)
# including pure Independents
pd.pivot_table(data, values=["avg_policy_support"],
               index=["party", "treatment_value", "treatment_frame"],
               aggfunc=['mean', 'sem'])

# SE for avg policy support is NaN: only one person with party I that was
# assigned treatment 2

mean                sem
                                       avg_policy_support avg_policy_support
party treatment_value treatment_frame                                       
D     0               No framing                 1.875000           0.161019
      1               Positive science           2.295455           0.088022
      2               Negative science           2.461538           0.126105
      3               Religious                  1.990000           0.122882
      4               Equity                     2.029412           0.142009
      5               Efficiency                 2.222222           0.106736
      6               Secular                    2.078947           0.156549
I     0               No framing                 1.300000           0.382426
      1               Positive science           0.812500           0.344223
      2               Negative science           1.750000                NaN
      3               Religious                  1.625000           0.179699
      4               Equity                     2.000000           0.367990
      5               Efficiency                 1.937500           0.239745
      6               Secular                    2.062500           0.213478
R     0               No framing                 0.750000           0.520416
      1               Positive science           1.571429           0.403535
      2               Negative science           0.437500           0.437500
      3               Religious                  1.375000           0.462106
      4               Equity                     1.666667           0.228218
      5               Efficiency                 1.166667           0.481606
      6               Secular                    1.500000           0.270031

In [15]:
# excluding pure Independents
pd.pivot_table(data.loc[data["party"] != "I"], values=["avg_policy_support"],
               index=["party", "treatment_value", "treatment_frame"],
               aggfunc=['mean', 'sem'])

mean                sem
                                       avg_policy_support avg_policy_support
party treatment_value treatment_frame                                       
D     0               No framing                 1.875000           0.161019
      1               Positive science           2.295455           0.088022
      2               Negative science           2.461538           0.126105
      3               Religious                  1.990000           0.122882
      4               Equity                     2.029412           0.142009
      5               Efficiency                 2.222222           0.106736
      6               Secular                    2.078947           0.156549
R     0               No framing                 0.750000           0.520416
      1               Positive science           1.571429           0.403535
      2               Negative science           0.437500           0.437500
      3               Religious                  1.375000           0.462106
      4               Equity                     1.666667           0.228218
      5               Efficiency                 1.166667           0.481606
      6               Secular                    1.500000           0.270031

In [16]:
# possible covariates: political party, political views, scientific confidence,
# religious, economic reasoning

# scientific confidence, political views each contain one NaN value

# party ID
party_id = data.loc[:, "party_id.1"]
data["party_cov"] = party_id.fillna(data["party_id.2"]).fillna(data["QID74"])
data.party_cov = data.party_cov.astype(int)

In [17]:
# for now creating indicator variable per treatment condition
treat_data = pd.get_dummies(data, columns=["treatment_value"])

In [18]:
# pretest response for control (in [0, 3])
treat_data["pre_avg_policy_support"] = treat_data[['GasTax', 'CarbTax', 'Treaty',
                                                   'RegCarb']].mean(axis=1)

In [19]:
# from code on shared doc
def lin_estimator_mult_treat(data, y_var, treatment_vars, covariates):
    """
    Inputs:
        data: pandas dataframe containing all x and y columns
        y_var: name of y variable
        treatment_vars: 
        covariates: list of string names of covariate

    Returns: tuple with Lin estimator model, formula for model
    """
    # Demean the covariates
    for cov in covariates:
        data[cov + '_demeaned'] = data[cov].dropna() - data[cov].dropna().mean()

    # Define the regression formula
    # Include each treatment indicator
    treatments_formula = " + ".join(treatment_vars)

    # Include each interaction term (automatically includes individual covariates)
    interactions = []
    for treatment in treatment_vars:
        for cov in covariates:
            interactions.append(f"{cov+ '_demeaned'}*{treatment}")

    interactions_formula = " + ".join(interactions)

    # Full formula -- include any other control(s)
    formula = f"{y_var} ~ {treatments_formula} + {interactions_formula}"

    # Fit the regression model and save results object
    model = smf.ols(formula, data=data).fit()

    # Return results object with robust covariance type
    return model.get_robustcov_results(cov_type="HC3"), formula

In [20]:
# drop treatment 0 (no frame condition) as reference level
# need to ask about this
treatment_vars = [f"treatment_value_{i}" for i in range(1, 7)]

# leaving political_views out for now because of multicollinearity with party_cov
# treat pre-test response like covariate
covariates = ["Religiosity", "ScientificConfidence", "Economic_Reasoning",
              "party_cov", "pre_avg_policy_support"]

y_var = "avg_policy_support"

model1_results, model1_formula = lin_estimator_mult_treat(treat_data, y_var,
                                                          treatment_vars,
                                                          covariates)

In [21]:
model1_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     avg_policy_support   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.853
Method:                 Least Squares   F-statistic:                     64.24
Date:                Fri, 26 Apr 2024   Prob (F-statistic):           9.80e-74
Time:                        18:21:23   Log-Likelihood:                -9.0159
No. Observations:                 185   AIC:                             102.0
Df Residuals:                     143   BIC:                             237.3
Df Model:                          41                                         
Covariance Type:                  HC3                                         
=============================================================================================================================
                                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------
Intercept                                                     1.7850      0.046     38.807      0.000       1.694       1.876
treatment_value_1[T.True]                                     0.1084      0.068      1.599      0.112      -0.026       0.242
treatment_value_2[T.True]                                     0.1749      0.125      1.404      0.162      -0.071       0.421
treatment_value_3[T.True]                                     0.1178      0.086      1.366      0.174      -0.053       0.288
treatment_value_4[T.True]                                     0.0548      0.070      0.783      0.435      -0.084       0.193
treatment_value_5[T.True]                                     0.0724      0.077      0.946      0.346      -0.079       0.224
treatment_value_6[T.True]                                     0.0442      0.179      0.246      0.806      -0.310       0.399
Religiosity_demeaned                                          0.1178      0.038      3.108      0.002       0.043       0.193
Religiosity_demeaned:treatment_value_1[T.True]               -0.1671      0.080     -2.077      0.040      -0.326      -0.008
Religiosity_demeaned:treatment_value_2[T.True]               -0.0353      0.099     -0.358      0.721      -0.230       0.160
Religiosity_demeaned:treatment_value_3[T.True]               -0.0597      0.055     -1.087      0.279      -0.168       0.049
Religiosity_demeaned:treatment_value_4[T.True]               -0.2010      0.084     -2.382      0.019      -0.368      -0.034
Religiosity_demeaned:treatment_value_5[T.True]               -0.1003      0.071     -1.419      0.158      -0.240       0.039
Religiosity_demeaned:treatment_value_6[T.True]               -0.2087      0.205     -1.018      0.310      -0.614       0.197
ScientificConfidence_demeaned                                 0.0653      0.073      0.897      0.371      -0.079       0.209
ScientificConfidence_demeaned:treatment_value_1[T.True]      -0.0286      0.143     -0.201      0.841      -0.310       0.253
ScientificConfidence_demeaned:treatment_value_2[T.True]      -0.3672      0.318     -1.154      0.250      -0.996       0.262
ScientificConfidence_demeaned:treatment_value_3[T.True]      -0.0089      0.115     -0.077      0.938      -0.237       0.219
ScientificConfidence_demeaned:treatment_value_4[T.True]      -0.1504      0.121     -1.243      0.216      -0.390       0.089
ScientificConfidence_demeaned:treatment_value_5[T.True]      -0.1246      0.220     -0.566      0.572      -0.560       0.311
ScientificConfidence_demeaned:treatment_value_6[T.True]       0.0702      0.164      0.428      0.670      -0.254       0.395
Economic_Reasoning_demeaned                                

**2. Does the framing that is best for people on average statistically outperform the control?**

Procedure for estimating effect of policy that is best on average:

- Split the data into two random folds.
- In each fold, using separate regression adjusted estimates, determine which treatment had the largest treatment effect. 
- Create a new variable which is an indicator for being in the best condition as determined by the opposite fold (i.e., if an observation is in fold 1, is it in the condition with the highest treatment effect in fold 2?). 
- Estimate the treatment effect of being in this condition as compared to the control using the regression-adjusted estimator. 

In [ ]:
# k = 2

**3. Does the best personalized assignment (i.e., where we give everyone the framing we think is best for them) outperform the framing that is best on average?**

- Use the same random folds, and the same best on average treatment from the previous part. 
- In each fold, fit a random forest under each treatment condition (so there will be 7 random forests in each fold). 
- For each observation, predict outcomes under each of the treatment conditions using the random forests from the opposite fold. 
- Create a new variable which is an indicator for being in the best personalized condition as determined by the opposite fold. 
- Estimate the treatment effect of being in this condition as compared to the control using the regression-adjusted estimator. 